# 今天学习Detectron2！

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
from detectron2.utils.logger import setup_logger
setup_logger()

<Logger detectron2 (DEBUG)>

import some common libraries

In [4]:
import numpy as np
import cv2

import some common detectron2 utilities

In [5]:
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

We first download a random image from the COCO dataset:

In [6]:
!wget http://images.cocodataset.org/val2017/000000439715.jpg -O ./pics/input.jpg

--2019-10-24 08:25:58--  http://images.cocodataset.org/val2017/000000439715.jpg
Resolving images.cocodataset.org (images.cocodataset.org)... 52.217.0.164
Connecting to images.cocodataset.org (images.cocodataset.org)|52.217.0.164|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 209222 (204K) [image/jpeg]
Saving to: './pics/input.jpg'

./pics/input.jpg    100%[===================>] 204.32K   277KB/s    in 0.7s    

2019-10-24 08:26:00 (277 KB/s) - './pics/input.jpg' saved [209222/209222]



显示图片，此处调用opencv。
- cv2.waitkey(delaytime)$\rightarrow$returnvalue
  - 在delaytime时间内,按键盘, 返回所按键的ASCII值;若未在delaytime时间内按任何键, 返回-1; 其中,dalaytime单位为ms;
  - 当delaytime为0时,表示forever,永不退回.

In [7]:
im = cv2.imread("./pics/input.jpg")
WINDOW_NAME="COCO pic"
cv2.namedWindow(WINDOW_NAME, cv2.WINDOW_NORMAL)
cv2.imshow(WINDOW_NAME,im)
cv2.waitKey(0)
cv2.destroyAllWindows()

Then, we create a detectron2 config and a detectron2 `DefaultPredictor` to run inference on this image.

In [8]:
cfg=get_cfg()
cfg.merge_from_file("../configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST=0.5 # set threshold for this model
cfg.MODEL.WEIGHTS="detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl"
predictor = DefaultPredictor(cfg)
outputs=predictor(im)

WARNING [10/24 08:26:10 d2.config.compat]: Config '../configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml' has no VERSION. Assuming it to be compatible with latest v2.


In [9]:
outputs["instances"].pred_classes
outputs["instances"].pred_boxes

Boxes(tensor([[126.6035, 244.8977, 459.8291, 480.0000],
        [251.1083, 157.8127, 338.9731, 413.6379],
        [114.8496, 268.6864, 148.2352, 398.8111],
        [  0.8217, 281.0327,  78.6073, 478.4210],
        [ 49.3954, 274.1229,  80.1545, 342.9808],
        [561.2248, 271.5816, 596.2755, 385.2552],
        [385.9072, 270.3125, 413.7130, 304.0397],
        [515.9295, 278.3744, 562.2792, 389.3802],
        [335.2409, 251.9167, 414.7491, 275.9375],
        [350.9300, 269.2060, 386.0984, 297.9081],
        [331.6292, 230.9996, 393.2759, 257.2009],
        [510.7349, 263.2656, 570.9865, 295.9194],
        [409.0841, 271.8646, 460.5582, 356.8722],
        [506.8766, 283.3257, 529.9404, 324.0392],
        [594.5663, 283.4820, 609.0577, 311.4124]], device='cuda:0'))

We can use `Visualizer` to draw the predictions on the image

In [10]:
v=Visualizer(im[:,:,::-1],MetadataCatalog.get(cfg.DATASETS.TRAIN[0]),scale=1.2)
v=v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2.imshow("WINDOW_NAME",v.get_image()[:,:,::-1])
cv2.waitKey(0)
cv2.destroyAllWindows()

## 电脑卡住后重启，出现PCIe Bus error

运行`cv2.imshow()`后没有跟`cv2.waitKey()`，结果电脑就卡住了，*其实我也不知道是不是`cv2.waitKey()`的原因，反正是重启了*

重启后分辨率出现问题，运行`nvidia-smi`说我没安装nvidia驱动，莫名其妙！

我`alt`+`ctrl`+`F6`进入命令行界面，不停的报错*PCIe Bus error*

我重启发现并没有改观，就重装了nvidia驱动，然后重启，分辨率正常了，但*PCIe Bus error*还在

网上搜到一般的解决方法都是[pci=nomsi](https://askubuntu.com/questions/911296/pcie-bus-error-ubuntu-16-04)方法,这种方法没交代error原因，也不根据错误代号来debug，我觉得不靠谱

[caya](https://www.cnblogs.com/caya-yuan/p/10604519.html)的博客说明了寻找PCIe Bus error的方法。另外，[小默_Allent](https://blog.csdn.net/l461269717/article/details/85133751)的博客介绍了PCI及lspci命令。
- PCI是Peripheral Component Interconnect（外围设备互联）的简称，是普遍使用在桌面及更大型的计算机上的外设总线。
- lspci 是一个用来显示系统中所有PCI总线设备或连接到该总线上的所有设备的工具。
- grep 指令用于查找内容包含指定的范本样式的文件，如果发现某文件的内容符合所指定的范本样式，预设 grep 指令会把含有范本样式的那一列显示出来。若不指定任何文件名称，或是所给予的文件名为 -，则 grep 指令会从标准输入设备读取数据。

当前错误为：

**pcieport 0000:00:1b.0: PCIe Bus Error: severity=Corrected, type=Physical Layer, (Receiver ID) device [8086:a2ed] error status/mask=00000001/00002000**

查找device [8086：a2ed]

输出:

- PCI bridge 是一种连接多个总线(Bus）的特殊PCI外设
- 根据输出来看，[8086:a2ed]似乎连接到00:1b.0 PCI bridge，进而查找00:1b.0(*理解不太透彻，暂且这么认为*）

输出: 

或者在PCI树状接口图中查找:

输出:

可以看到1b.0连接到02设备，继续查找:

输出:

根据树状图，应该是02:00.0有问题，看起来和无限网有关，我把无线网关了，果然立刻停止报错。

但这终究不是长久之计，似乎需要重装无线网卡驱动。